# Mining OLX Website

This is an experiment for my initial study in web scrapping, using libraries like requests and bs4 we will find the most important informations, like price, numbers of baths, and more.

In [1]:
import pandas as pd
import numpy as np
import requests_html
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import re

In case i will search only through São Paulo, i got this links before, manually, but, if it have more links we can automize this collection

In [2]:
regioes_venda = ['https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-norte/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-oeste/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-sul/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/abcd/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/alphaville-e-tambore/imoveis/venda',
          'https://sp.olx.com.br/sao-paulo-e-regiao/outras-cidades/imoveis/venda']

regioes_aluguel = ['https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-norte/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-oeste/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/zona-sul/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/abcd/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/alphaville-e-tambore/imoveis/aluguel',
          'https://sp.olx.com.br/sao-paulo-e-regiao/outras-cidades/imoveis/aluguel']

df_a = pd.DataFrame()
df_v = pd.DataFrame()

## Mining Properties for sale

The first thing to do is get the to website, loop throug the 100 first pages (website limits to 100 pages), and search for links with the pattern **/imoveis/([^"]+)**, and then find other patterns, like **location:text**, **pricevalue:text** and storing this.

To prevent some erros, we use expections



In [ ]:
for reg in regioes_venda:
    print("Mineirando dados da Região: " + reg)
    for pagina in range(101):
        print('Pagina: '+str(pagina))
        try:
            session1 = HTMLSession()
            regiao = session1.get(str(reg)+'?o='+str(pagina))
            soup_regiao = BeautifulSoup(regiao.text, 'html.parser')
            links = re.findall( r'href="https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/([^"]+)"',str(soup_regiao))

            if(len(links)>5):
                for i in range(1,len(links)):
                    session2 = HTMLSession()
                    imovel = session1.get('https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/'+links[i])
                    soup_imovel = BeautifulSoup(imovel.text, 'html.parser')
                    
                    try:
                        cat = ['url',"address", 'neighbourhood', 'rent_value', 'City', 'CEP']

                        # Localização
                        try:
                            location_raw = re.findall(r'"location":{.*?\}', str(soup_imovel))[1]
                        except:
                            location_raw=[]
                            print('1')
                                                    
                        
                        try:
                            address = re.findall(r'"address":"([^"]+)"', str(location_raw))[0]
                        except:
                            address = "Na"
                            print('2')
                            
                        try:
                            neighbourhood = re.findall(r'"neighbourhood":"([^"]+)"', str(location_raw))[0]
                        except:
                            neighbourhood="Na"
                            print('3')
                        
                        # Rent Value
                        try:
                            value = re.findall(r'"priceValue":"([^"]*)"', str(soup_imovel))[0]
                        except:
                            value=-5000
                            print('4')
                        
                        try:
                            city = re.findall(r'"municipality":"([^"]*)"', str(soup_imovel))[0]
                        except:
                            city='missing'
                            print('8')
                            
                        try:
                            cep = re.findall(r'"zipcode":"([^"]*)"', str(soup_imovel))[0]
                        except:
                            cep='99999999'
                            print('9')
                        
                        val = ['https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/'+links[i] ,address, neighbourhood,value,
                              city, cep]


                        #Detalhes Apartamento
                        try:
                            detail_raw = re.findall(r'{"name":.*?\}', str(soup_imovel))
                        except:
                            detail_raw=[]
                            print('5')

                        for det in range(len(detail_raw)):
                            try:
                                name_cat = re.findall(r'"name":"([^"]*)"', str(detail_raw[det]))[0]
                            except:
                                name_cat = "Na"
                                print('5')
                                
                            try:
                                aux = re.findall(r'"value":"([^"]*)"', str(detail_raw[det]))[0]
                            except:
                                aux = "Na"
                                print('6')
                                
                            cat.append(name_cat)
                            val.append(aux)
                        teste = pd.DataFrame(data = [val], columns=cat)
                        df_v = pd.concat([df_v, teste])

                    except Exception as inst:
                        
                        print("O imovel: "+links[i] + " Foi excluido pelo dono")
                        print(' ------------------------------ ')
        except:
            print("Erro regiao")
            print(str(reg)+'?o='+str(pagina))

Mineirando dados da Região: https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/venda
Pagina: 0
4
1
2
3
4
8
Erro regiao
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/venda?o=0
Pagina: 1


## Mining Properties for rent

The algorithms here are the some of properties rent, just changing the initial link

In [3]:
for reg in regioes_aluguel:
    print("Mineirando dados da Região: " + reg)
    for pagina in range(101):
        print('Pagina: '+str(pagina))
        try:
            session1 = HTMLSession()
            regiao = session1.get(str(reg)+'?o='+str(pagina))
            soup_regiao = BeautifulSoup(regiao.text, 'html.parser')
            links = re.findall( r'href="https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/([^"]+)"',str(soup_regiao))

            if(len(links)>5):
                for i in range(1,len(links)):
                    session2 = HTMLSession()
                    imovel = session1.get('https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/'+links[i])
                    soup_imovel = BeautifulSoup(imovel.text, 'html.parser')
                    
                    try:
                        cat = ['url',"address", 'neighbourhood', 'value',  'City', 'CEP']

                        # Localização
                        try:
                            location_raw = re.findall(r'"location":{.*?\}', str(soup_imovel))[1]
                        except:
                            location_raw=[]
                            print('1')
                                                    
                        
                        try:
                            address = re.findall(r'"address":"([^"]+)"', str(location_raw))[0]
                        except:
                            address = "Na"
                            print('2')
                            
                        try:
                            neighbourhood = re.findall(r'"neighbourhood":"([^"]+)"', str(location_raw))[0]
                        except:
                            neighbourhood="Na"
                            print('3')
                        
                        # Valor Aluguel
                        try:
                            value = re.findall(r'"priceValue":"([^"]*)"', str(soup_imovel))[0]
                        except:
                            value=-5000
                            print('4')
                            
                        try:
                            city = re.findall(r'"municipality":"([^"]*)"', str(soup_imovel))[0]
                        except:
                            city='missing'
                            print('8')
                            
                        try:
                            cep = re.findall(r'"zipcode":"([^"]*)"', str(soup_imovel))[0]
                        except:
                            cep='99999999'
                            print('9')
                        
                        val = ['https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/'+links[i] ,address, neighbourhood,value,
                               city, cep]


                        #Detalhes Apartamento
                        try:
                            detail_raw = re.findall(r'{"name":.*?\}', str(soup_imovel))
                        except:
                            detail_raw=[]
                            print('5')

                        for det in range(len(detail_raw)):
                            try:
                                name_cat = re.findall(r'"name":"([^"]*)"', str(detail_raw[det]))[0]
                            except:
                                name_cat = "Na"
                                print('5')
                                
                            try:
                                aux = re.findall(r'"value":"([^"]*)"', str(detail_raw[det]))[0]
                            except:
                                aux = "Na"
                                print('6')
                                
                            cat.append(name_cat)
                            val.append(aux)
                        teste = pd.DataFrame(data = [val], columns=cat)
                        df_a = pd.concat([df_a, teste])

                    except Exception as inst:
                        
                        print("O Aluguel: "+links[i] + " Foi excluido pelo dono")
                        print(' ------------------------------ ')
        except:
            print("Erro regiao")
            print(str(reg)+'?o='+str(pagina))

Mineirando dados da Região: https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel
Pagina: 0
1
2
3
4
8
2
1
2
3
4
8
9
1
2
3
4
8
9
1
2
3
4
8
9
1
2
3
4
8
9
4
Pagina: 1
4
2
4
Pagina: 2
4
4
Pagina: 3
4
4
4
Pagina: 4
2
4
4
4
2
4
4
4
Pagina: 5
4
4
1
2
3
4
8
9
4
Pagina: 6
2
2
2
Pagina: 7
4
4
2
Pagina: 8
4
2
4
4
4
Pagina: 9
4
2
4
2
Pagina: 10
4
Pagina: 11
4
4
4
Pagina: 12
4
4
4
4
4
2
Pagina: 13
4
4
4
4
2
4
4
1
2
3
4
8
Pagina: 14
2
4
4
2
4
4
Pagina: 15
4
Pagina: 16
4
4
1
2
3
4
8
4
Pagina: 17
1
2
3
4
8
9
2
Pagina: 18
2
2
4
4
Pagina: 19
4
4
4
2
4
2
Pagina: 20
4
2
4
4
Pagina: 21
4
2
4
4
Pagina: 22
2
2
2
1
2
3
4
8
1
2
3
4
8
9
1
2
3
4
8
9
Pagina: 23
1
2
3
4
8
9
2
1
2
3
4
8
1
2
3
4
8
9
Pagina: 24
2
2
2
4
1
2
3
4
8
9
Pagina: 25
2
2
1
2
3
4
8
1
2
3
4
8
4
Pagina: 26
4
4
4
Pagina: 27
1
2
3
4
8
2
Pagina: 28
1
2
3
4
8
Pagina: 29
1
2
3
4
8
4
1
2
3
4
8
Pagina: 30
4
4
Pagina: 31
2
4
4
4
Pagina: 32
1
2
3
4
8
Pagina: 33
4
4
4
4
Pagina: 34
4
1
2
3
4
8
4
Pagina: 35
4
Pagina: 36
4
4
4
2
4
4
4
1
2
3
4
8
Pag

4
2
Pagina: 25
1
2
3
4
8
2
Pagina: 26
2
Pagina: 27
2
Pagina: 28
Pagina: 29
2
Pagina: 30
Pagina: 31
4
2
Pagina: 32
4
4
4
Pagina: 33
4
1
2
3
4
8
2
Pagina: 34
2
Pagina: 35
Pagina: 36
2
Pagina: 37
2
Pagina: 38
Pagina: 39
Pagina: 40
1
2
3
4
8
2
4
4
Pagina: 41
1
2
3
4
8
4
4
4
Pagina: 42
1
2
3
4
8
2
Pagina: 43
4
1
2
3
4
8
Pagina: 44
4
4
4
Pagina: 45
1
2
3
4
8
Pagina: 46
4
2
Pagina: 47
2
Pagina: 48
4
1
2
3
4
8
2
Pagina: 49
4
Pagina: 50
2
4
Pagina: 51
1
2
3
4
8
1
2
3
4
8
4
1
2
3
4
8
Pagina: 52
Pagina: 53
2
1
2
3
4
8
Pagina: 54
2
2
Pagina: 55
4
Pagina: 56
Pagina: 57
2
2
4
4
2
2
Pagina: 58
4
4
Pagina: 59
1
2
3
4
8
1
2
3
4
8
1
2
3
4
8
4
Pagina: 60
4
4
Pagina: 61
4
4
1
2
3
4
8
1
2
3
4
8
2
4
Pagina: 62
4
4
Pagina: 63
4
1
2
3
4
8
Pagina: 64
1
2
3
4
8
1
2
3
4
8
2
2
Pagina: 65
2
Pagina: 66
4
4
4
2
2
Pagina: 67
4
Pagina: 68
2
4
4
4
Pagina: 69
2
4
Pagina: 70
4
Pagina: 71
4
4
Pagina: 72
2
4
Pagina: 73
4
Pagina: 74
4
Pagina: 75
Pagina: 76
2
4
4
2
2
Pagina: 77
1
2
3
4
8
1
2
3
4
8
4
Pagina: 78
Pagina: 79
1
2

In [4]:
df_a.to_csv('data/PropertyRent.csv')
df_v.to_csv('data/PropertySale.csv')